In [1]:
# Clone the entire repo.
!git clone -l -s git://github.com/Idan707/Kaggle_TMDB_Box_Office_Prediction.git cloned-repo
%cd cloned-repo
# !ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 159 (delta 86), reused 70 (delta 29), pack-reused 0
Receiving objects: 100% (159/159), 17.99 MiB | 23.71 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/cloned-repo


In [2]:
#pip install pandas==0.25
pip install catboost

     |████████████████████████████████| 63.9MB 67kB/s 


# Import & Load

In [0]:
# imports
import pandas as pd
import  numpy as np
import ast
import json
import gensim
import datetime
import ast
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.preprocessing import MultiLabelBinarizer ,normalize
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
import lightgbm as lgb
import catboost as cat
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import GroupKFold
import xgboost as xgb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

train = pd.read_csv('/content/cloned-repo/Data/train.csv',
                    parse_dates=['release_date'])
test = pd.read_csv('/content/cloned-repo/Data/test.csv',
                    parse_dates=['release_date'])

# Date Fix
train['release_date'] = train['release_date'].mask(train['release_date'].dt.year > 2017, 
                                                   train['release_date'] - pd.offsets.DateOffset(years=100))
test['release_date'] = test['release_date'].mask(test['release_date'].dt.year > 2017, 
                                                   test['release_date'] - pd.offsets.DateOffset(years=100))

test['revenue'] = 0

mlb = MultiLabelBinarizer()

In [0]:
train.loc[train['id'] == 16,'revenue'] = 192864         
train.loc[train['id'] == 90,'budget'] = 30000000                  
train.loc[train['id'] == 118,'budget'] = 60000000       
train.loc[train['id'] == 149,'budget'] = 18000000       
train.loc[train['id'] == 313,'revenue'] = 12000000       
train.loc[train['id'] == 451,'revenue'] = 12000000      
train.loc[train['id'] == 464,'budget'] = 20000000       
train.loc[train['id'] == 470,'budget'] = 13000000       
train.loc[train['id'] == 513,'budget'] = 930000         
train.loc[train['id'] == 797,'budget'] = 8000000        
train.loc[train['id'] == 819,'budget'] = 90000000       
train.loc[train['id'] == 850,'budget'] = 90000000       
train.loc[train['id'] == 1007,'budget'] = 2              
train.loc[train['id'] == 1112,'budget'] = 7500000       
train.loc[train['id'] == 1131,'budget'] = 4300000        
train.loc[train['id'] == 1359,'budget'] = 10000000       
train.loc[train['id'] == 1542,'budget'] = 1             
train.loc[train['id'] == 1570,'budget'] = 15800000       
train.loc[train['id'] == 1571,'budget'] = 4000000        
train.loc[train['id'] == 1714,'budget'] = 46000000       
train.loc[train['id'] == 1721,'budget'] = 17500000       
train.loc[train['id'] == 1865,'revenue'] = 25000000      
train.loc[train['id'] == 1885,'budget'] = 12             
train.loc[train['id'] == 2091,'budget'] = 10             
train.loc[train['id'] == 2268,'budget'] = 17500000       
train.loc[train['id'] == 2491,'budget'] = 6              
train.loc[train['id'] == 2602,'budget'] = 31000000       
train.loc[train['id'] == 2612,'budget'] = 15000000       
train.loc[train['id'] == 2696,'budget'] = 10000000      
train.loc[train['id'] == 2801,'budget'] = 10000000       
train.loc[train['id'] == 335,'budget'] = 2 
train.loc[train['id'] == 348,'budget'] = 12
train.loc[train['id'] == 470,'budget'] = 13000000 
train.loc[train['id'] == 513,'budget'] = 1100000
train.loc[train['id'] == 640,'budget'] = 6 
train.loc[train['id'] == 696,'budget'] = 1
train.loc[train['id'] == 797,'budget'] = 8000000 
train.loc[train['id'] == 850,'budget'] = 1500000
train.loc[train['id'] == 1199,'budget'] = 5 
train.loc[train['id'] == 1282,'budget'] = 9              
train.loc[train['id'] == 1347,'budget'] = 1
train.loc[train['id'] == 1755,'budget'] = 2
train.loc[train['id'] == 1801,'budget'] = 5
train.loc[train['id'] == 1918,'budget'] = 592 
train.loc[train['id'] == 2033,'budget'] = 4
train.loc[train['id'] == 2118,'budget'] = 344 
train.loc[train['id'] == 2252,'budget'] = 130
train.loc[train['id'] == 2256,'budget'] = 1 
train.loc[train['id'] == 2696,'budget'] = 10000000
test.loc[test['id'] == 3033,'budget'] = 250 
test.loc[test['id'] == 3051,'budget'] = 50
test.loc[test['id'] == 3084,'budget'] = 337
test.loc[test['id'] == 3224,'budget'] = 4  
test.loc[test['id'] == 3594,'budget'] = 25  
test.loc[test['id'] == 3619,'budget'] = 500  
test.loc[test['id'] == 3831,'budget'] = 3  
test.loc[test['id'] == 3935,'budget'] = 500  
test.loc[test['id'] == 4049,'budget'] = 995946 
test.loc[test['id'] == 4424,'budget'] = 3  
test.loc[test['id'] == 4460,'budget'] = 8  
test.loc[test['id'] == 4555,'budget'] = 1200000 
test.loc[test['id'] == 4624,'budget'] = 30 
test.loc[test['id'] == 4645,'budget'] = 500 
test.loc[test['id'] == 4709,'budget'] = 450 
test.loc[test['id'] == 4839,'budget'] = 7
test.loc[test['id'] == 3125,'budget'] = 25 
test.loc[test['id'] == 3142,'budget'] = 1
test.loc[test['id'] == 3201,'budget'] = 450
test.loc[test['id'] == 3222,'budget'] = 6
test.loc[test['id'] == 3545,'budget'] = 38
test.loc[test['id'] == 3670,'budget'] = 18
test.loc[test['id'] == 3792,'budget'] = 19
test.loc[test['id'] == 3881,'budget'] = 7
test.loc[test['id'] == 3969,'budget'] = 400
test.loc[test['id'] == 4196,'budget'] = 6
test.loc[test['id'] == 4221,'budget'] = 11
test.loc[test['id'] == 4222,'budget'] = 500
test.loc[test['id'] == 4285,'budget'] = 11
test.loc[test['id'] == 4319,'budget'] = 1
test.loc[test['id'] == 4639,'budget'] = 10
test.loc[test['id'] == 4719,'budget'] = 45
test.loc[test['id'] == 4822,'budget'] = 22
test.loc[test['id'] == 4829,'budget'] = 20
test.loc[test['id'] == 4969,'budget'] = 20
test.loc[test['id'] == 5021,'budget'] = 40 
test.loc[test['id'] == 5035,'budget'] = 1 
test.loc[test['id'] == 5063,'budget'] = 14 
test.loc[test['id'] == 5119,'budget'] = 2 
test.loc[test['id'] == 5214,'budget'] = 30 
test.loc[test['id'] == 5221,'budget'] = 50 
test.loc[test['id'] == 4903,'budget'] = 15
test.loc[test['id'] == 4983,'budget'] = 3
test.loc[test['id'] == 5102,'budget'] = 28
test.loc[test['id'] == 5217,'budget'] = 75
test.loc[test['id'] == 5224,'budget'] = 3 
test.loc[test['id'] == 5469,'budget'] = 20 
test.loc[test['id'] == 5840,'budget'] = 1 
test.loc[test['id'] == 5960,'budget'] = 30
test.loc[test['id'] == 6506,'budget'] = 11 
test.loc[test['id'] == 6553,'budget'] = 280
test.loc[test['id'] == 6561,'budget'] = 7
test.loc[test['id'] == 6582,'budget'] = 218
test.loc[test['id'] == 6638,'budget'] = 5
test.loc[test['id'] == 6749,'budget'] = 8 
test.loc[test['id'] == 6759,'budget'] = 50 
test.loc[test['id'] == 6856,'budget'] = 10
test.loc[test['id'] == 6858,'budget'] =  100
test.loc[test['id'] == 6876,'budget'] =  250
test.loc[test['id'] == 6972,'budget'] = 1
test.loc[test['id'] == 7079,'budget'] = 8000000
test.loc[test['id'] == 7150,'budget'] = 118
test.loc[test['id'] == 6506,'budget'] = 118
test.loc[test['id'] == 7225,'budget'] = 6
test.loc[test['id'] == 7231,'budget'] = 85
test.loc[test['id'] == 5222,'budget'] = 5
test.loc[test['id'] == 5322,'budget'] = 90
test.loc[test['id'] == 5350,'budget'] = 70
test.loc[test['id'] == 5378,'budget'] = 10
test.loc[test['id'] == 5545,'budget'] = 80
test.loc[test['id'] == 5810,'budget'] = 8
test.loc[test['id'] == 5926,'budget'] = 300
test.loc[test['id'] == 5927,'budget'] = 4
test.loc[test['id'] == 5986,'budget'] = 1
test.loc[test['id'] == 6053,'budget'] = 20
test.loc[test['id'] == 6104,'budget'] = 1
test.loc[test['id'] == 6130,'budget'] = 30
test.loc[test['id'] == 6301,'budget'] = 150
test.loc[test['id'] == 6276,'budget'] = 100
test.loc[test['id'] == 6473,'budget'] = 100
test.loc[test['id'] == 6842,'budget'] = 30

# Feature Engineering 

In [21]:
def convert_string_to_list(strVal):
    if type(strVal) is not str:
        return  []
    else:
        return ast.literal_eval(strVal)
    
def format_dict_column_and_extract_names(strVal, col="name"):
    listOfItems = convert_string_to_list(strVal)

    return list(map(lambda x: x[col], listOfItems))

def add_x_length_column(df, col):
    df[col + '_size'] = df[col].apply(lambda x: len(x))
    
    return df

def extract_genres(df):
    df['genres'] = df['genres'].apply(format_dict_column_and_extract_names)
    df = add_x_length_column(df, col='genres')
    temp = mlb.fit_transform(df.pop('genres'))
    temp_df = pd.DataFrame(temp, columns=list(map(lambda x: 'genres'+'_'+x,mlb.classes_)), 
                              )#index=temp.index)
    #print('extract_genres temp df shape: ', temp_df.shape)
    df = pd.merge(df, temp_df, how='inner', on=df.id)
    
    return df

def extract_common(df, col, limit, apply_dict_to_col=True):
    if apply_dict_to_col == True:
        df[col] = df[col].apply(format_dict_column_and_extract_names)
        
    companiesCount = df[col].apply(pd.Series).stack().value_counts().sort_values(axis=0, ascending=False)
    companiesToKeep = companiesCount[:limit].keys()
    
    add_x_length_column(df, col)
    df[col] = df[col].apply(lambda x: list(filter(lambda i: i in companiesToKeep, x)))
    
    temp = mlb.fit_transform(df.pop(col))
    temp_df = pd.DataFrame(temp, columns=list(map(lambda x: col+'_'+x,mlb.classes_)), 
                              )#index=temp.index)
    #print('extract_common temp df shape: ', temp_df.shape)
    #print('temp index: ',temp_df.index)
    df = pd.merge(df, temp_df, how='inner', left_index=True, right_index=True)

    return df

def last_year_metrics(df, agg_col='release_date_Year', calc_col=['revenue','imdb_id','popularity','budget'], leg=1):
    
    temp = df.groupby([agg_col]).agg(last_period_mean_revenue = (calc_col[0], 'mean'),
                                     last_period_median_revenue = (calc_col[0], 'median'),
                                     last_period_movies_count = (calc_col[1], 'count'),
                                     last_period_mean_popularity = (calc_col[2], 'mean'),
                                     last_period_median_popularity = (calc_col[2], 'median'),
                                     last_period_mean_budget = (calc_col[3], 'mean'),
                                     last_period_median_budget = (calc_col[3], 'median')).shift(leg).reset_index()

    return pd.merge(df, temp, on=['release_date_Year'], how='left')

def days_diff_from_last_movie(df, col='release_date'):#see if there is a need in buckts
    df = df.sort_values(by=[col])
    df['days_diff_from_last_movie'] = df[col].diff().fillna(0).dt.days
    
    return df

def ratios(df):
    df['budget_to_runtime'] = df['budget']/df['runtime']
    df['budget_to_cast_crew_size'] = df['budget']/(df['cast_size'] + df['crew_size'])
    df['budget_to_popularity'] = df['budget']/df['popularity']
    
    return df

def add_datepart(df,col ='release_date'):
    df[col] = pd.to_datetime(df[col])
    df[col +'_Year']  = df[col].dt.year
    df[col +'_Month'] = df[col].dt.month
    df[col +'_day_of_month'] = df[col].dt.day
    df[col +'_week_day'] = df[col].dt.dayofweek
    df[col +'_is_quarter_start'] = df[col].dt.is_quarter_start
    df[col +'_is_quarter_start'] = df[col +'_is_quarter_start'].fillna(False)
    
    return df 

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
            
    return result

def extract_common_overview_words(df, col):
    df[col] = df[col].fillna('').astype(str).map(preprocess)
    df[col] = df[col].map(lambda x: list(set(x)))
    df = extract_common(df, col, limit=15, apply_dict_to_col=False)
    
    return df
  
def extract_gender(strVal, dict_key, dict_value):
    listOfItems = convert_string_to_list(strVal)

    Item = (list(filter(lambda lst: lst[dict_key] == dict_value, listOfItems)) or [None])[0]
    if type(Item) is dict:
        return Item['gender']
    else:
        return None
      
def add_gender(df, col, dict_key_in , dict_value_in):
    df[dict_key_in + '_' + str(dict_value_in) + '_gender'] = df[col].apply(extract_gender, args=[dict_key_in, dict_value_in])
    df[dict_key_in + '_' + str(dict_value_in) + '_gender'] = df[dict_key_in + '_' + str(dict_value_in) + '_gender'].fillna(2).astype('int64')
    
    return df

def run_main(train, test):

  test['train_test'] = 'test'  
  train['train_test'] = 'train'

  # work on full df
  print('working on full df..')
  df = pd.concat([train, test],axis=0, sort=True)
  df = extract_genres(df)
  df = extract_common(df, col='production_companies', limit=30)
  df = extract_common(df, col='production_countries', limit=20)
  df = extract_common(df, col='spoken_languages', limit=15)
  df = extract_common(df, col='Keywords', limit=15)
  
  df = add_gender(df, col='crew', dict_key_in='job', dict_value_in='Producer')
  df = add_gender(df, col='crew', dict_key_in='job', dict_value_in='Director')
  df = add_gender(df, col='cast', dict_key_in='order', dict_value_in=0)
  df = add_gender(df, col='cast', dict_key_in='order', dict_value_in=1)
  df = extract_common(df, col='cast', limit=15)
  df = extract_common(df, col='crew', limit=15)

  # fill missing values 1
  print('fill missing values 1...')
  df['cast_size'] = np.where(df['cast_size'] == 0, df['cast_size'].mean(), df['cast_size'])
  df['crew_size'] = np.where(df['crew_size'] == 0, df['crew_size'].mean(), df['crew_size'])
  
  # add datepart
  print('add datepart...')
  df = add_datepart(df,col = 'release_date') 

  # split for data leak prevention
  print('split for data leak prevention and run main...')
  train = df[df['train_test'] == 'train']
  test = df[df['train_test'] == 'test']

  train = last_year_metrics(train)
  train = days_diff_from_last_movie(train)
  train['revenue'] = np.where(train['revenue'] == 0, train['last_period_median_revenue'], train['revenue'])
  train['revenue'] = np.where(train['revenue'] == 0, train["revenue"].mean(), train['revenue'])
  train = extract_common_overview_words(train, col='overview')

  test = last_year_metrics(test)
  test = days_diff_from_last_movie(test)
  test['revenue'] = np.where(test['revenue'] == 0, test['last_period_median_revenue'], test['revenue'])
  test['revenue'] = np.where(test['revenue'] == 0, test["revenue"].mean(), test['revenue'])
  test = extract_common_overview_words(test, col='overview')

  df = pd.concat([train, test],axis=0, sort=True)

  # fill missing values 2
  print('fill missing values 2...')
  df['budget'] = np.where(df['budget'] == 0, df['last_period_median_budget'], df['budget'])
  df['budget'] = np.where(df['budget'] == 0, df["budget"].mean(), df['budget'])
  df['production_companies_size'] = np.where(df['production_companies_size'] == 0, df["production_companies_size"].mean(), df['production_companies_size'])
  df['production_countries_size'] = np.where(df['production_countries_size'] == 0, df["production_countries_size"].mean(), df['production_countries_size'])
  df['Keywords_size'] = np.where(df['Keywords_size'] == 0, df["Keywords_size"].mean(), df['Keywords_size'])

  # run ratios
  print('run ratios...')
  df = ratios(df)

  # fill missing values 3
  print('fill missing values 3...')
  df = df.replace([np.inf, -np.inf], np.nan)
  df['budget_to_runtime'] = np.where(df['budget_to_runtime'] == np.nan, df["budget_to_runtime"].mean(), df['budget_to_runtime'])
  df['belongs_to_collection_bool'] = np.where(df['belongs_to_collection'].isna(), 0, 1)
  df.drop(['belongs_to_collection'], axis=1, inplace=True)

  # counting the numbers of words in the movie title 
  print('counting the numbers of words in the movie title..')
  df['original_title_words'] = df.original_title.str.split()
  df['original_title_words'] = df['original_title_words'].apply(lambda x: len(x))

  df = pd.get_dummies(df, prefix=['job_Producer_gender', 'job_Director_gender', 'order_0_gender', 'order_1_gender'], 
                          columns=['job_Producer_gender', 'job_Director_gender', 'order_0_gender', 'order_1_gender'])

  # removing highly correlated features
  print('removing highly correlated features...')
  df.drop(["crew_Bob Weinstein", "release_date_Year"], inplace=True, axis=1)

  # normalize features
  scaler = preprocessing.MinMaxScaler()
  df["budget_to_popularity"] = scaler.fit_transform(df[["budget_to_popularity"]])

  return df

df = run_main(train, test)
df.head()

working on full df..
fill missing values 1...
add datepart...
split for data leak prevention and run main...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.


fill missing values 2...
run ratios...
fill missing values 3...
counting the numbers of words in the movie title..
removing highly correlated features...


,Keywords_aftercreditsstinger,Keywords_based on novel,Keywords_biography,Keywords_duringcreditsstinger,Keywords_dystopia,Keywords_friendship,Keywords_independent film,Keywords_love,Keywords_murder,Keywords_police,Keywords_revenge,Keywords_sex,Keywords_size,Keywords_sport,Keywords_violence,Keywords_woman director,budget,cast_Bruce Willis,cast_Christopher Walken,cast_J.K. Simmons,cast_John Goodman,cast_Julianne Moore,cast_Liam Neeson,cast_Matt Damon,cast_Morgan Freeman,cast_Nicolas Cage,cast_Robert De Niro,cast_Robin Williams,cast_Samuel L. Jackson,cast_Steve Buscemi,cast_Sylvester Stallone,cast_Willem Dafoe,cast_size,crew_Avy Kaufman,crew_Deborah Aquila,crew_Francine Maisler,crew_Hans Zimmer,crew_Harvey Weinstein,crew_James Horner,crew_James Newton Howard,crew_Jerry Goldsmith,crew_Kerry Barden,crew_Luc Besson,crew_Mary Vernieu,crew_Robert Rodriguez,crew_Steven Spielberg,crew_Tricia Wood,crew_size,days_diff_from_last_movie,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Foreign,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western,genres_size,homepage,id,imdb_id,key_0,last_period_mean_budget,last_period_mean_popularity,last_period_mean_revenue,last_period_median_budget,last_period_median_popularity,last_period_median_revenue,last_period_movies_count,original_language,original_title,overview_family,overview_father,overview_film,overview_finds,overview_friends,overview_help,overview_life,overview_love,overview_size,overview_story,overview_time,overview_wife,overview_woman,overview_world,overview_year,overview_years,overview_young,popularity,poster_path,production_companies_Amblin Entertainment,production_companies_BBC Films,production_companies_Canal+,production_companies_Columbia Pictures,production_companies_Columbia Pictures Corporation,production_companies_Dimension Films,production_companies_DreamWorks SKG,production_companies_Dune Entertainment,production_companies_Fox 2000 Pictures,production_companies_Fox Searchlight Pictures,production_companies_Hollywood Pictures,production_companies_Lionsgate,production_companies_Metro-Goldwyn-Mayer (MGM),production_companies_Miramax Films,production_companies_New Line Cinema,production_companies_Orion Pictures,production_companies_Paramount Pictures,production_companies_Regency Enterprises,production_companies_Relativity Media,production_companies_StudioCanal,production_companies_Summit Entertainment,production_companies_Touchstone Pictures,production_companies_TriStar Pictures,production_companies_Twentieth Century Fox Film Corporation,production_companies_United Artists,production_companies_Universal Pictures,production_companies_Village Roadshow Pictures,production_companies_Walt Disney Pictures,production_companies_Warner Bros.,production_companies_Working Title Films,production_companies_size,production_countries_Australia,production_countries_Belgium,production_countries_Canada,production_countries_China,production_countries_Denmark,production_countries_France,production_countries_Germany,production_countries_Hong Kong,production_countries_India,production_countries_Ireland,production_countries_Italy,production_countries_Japan,production_countries_Mexico,production_countries_Netherlands,production_countries_Russia,production_countries_South Korea,production_countries_Spain,production_countries_Sweden,production_countries_United Kingdom,production_countries_United States of America,production_countries_size,release_date,release_date_Month,release_date_day_of_month,release_date_is_quarter_start,release_date_week_day,revenue,runtime,spoken_languages_,spoken_languages_Deutsch,spoken_languages_English,spoken_languages_Español,spoken_languages_Français,spoken_languages_Italiano,spoken_languages_Polski,spoken_languages_Português,spoken_languages_Pусский,spoken_languages_size,spoken_languages_العر

In [0]:
#import pandas_profiling
#report = pandas_profiling.ProfileReport(df)
#report.to_file("your_report.html")

# Modeling

In [0]:
model_df = df.copy()
# model_df['id'] =  model_df.index
model_df = model_df.drop(["homepage", "imdb_id", "original_language", "original_title", "poster_path", "release_date", "status", "tagline", "title"], axis=1)

train = model_df[model_df['train_test'] == 'train']
test = model_df[model_df['train_test'] == 'test']

#train_y = train[['revenue']]
#test_y = test[['revenue']]
train = train.drop(['train_test'],axis=1)
test = test.drop(['train_test'] ,axis=1)

# scale_train = StandardScaler().fit(train)
# scale_df = scale_train.transform(train)
# train = pd.DataFrame(scale_df,columns=train.columns)#.merge(train_y, left_index=True, right_index=True)

# scale_test = StandardScaler().fit(test)
# scale_df = scale_test.transform(test)
# test = pd.DataFrame(scale_df,columns=test.columns)#.merge(test_y, left_index=True, right_index=True)


def score(data, y):
    validation_res = pd.DataFrame(
    {"id": data["id"].values,
     "transactionrevenue": data["revenue"].values,
     "predictedrevenue": np.expm1(y)}) 

    print(validation_res.head())

    validation_res = validation_res.groupby("id")["transactionrevenue", "predictedrevenue"].sum().reset_index()
    
    print(validation_res.head())
    return  np.sqrt(mean_squared_error((np.log1p(validation_res["transactionrevenue"].values)), #np.log1p
                                     (np.log1p(validation_res["predictedrevenue"].values)))) #np.log1p
    
class KFoldValidation():
    def __init__(self, data, n_splits=5):
        unique_vis = np.array(sorted(data['id'].astype(str).unique()))
        folds = GroupKFold(n_splits)
        ids = np.arange(data.shape[0])
        
        self.fold_ids = []
        for trn_vis, val_vis in folds.split(X=unique_vis, y=unique_vis, groups=unique_vis):
            self.fold_ids.append([
                    ids[data['id'].astype(str).isin(unique_vis[trn_vis])],
                    ids[data['id'].astype(str).isin(unique_vis[val_vis])]
                ])
            
    def validate(self, train, test, features, model, name="", prepare_stacking=False, 
                 fit_params={"early_stopping_rounds": 500, "verbose": 100, "eval_metric": "rmse"}):
        model.FI = pd.DataFrame(index=features)
        full_score = 0
        
        if prepare_stacking:
            test[name] = 0
            train[name] = np.NaN
        
        for fold_id, (trn, val) in enumerate(self.fold_ids):
            devel = train[features].iloc[trn]
            y_devel = np.log1p(train["revenue"].iloc[trn])
            valid = train[features].iloc[val]
            y_valid = np.log1p(train["revenue"].iloc[val])
                       
            print("Fold ", fold_id, ":")
            model.fit(devel, y_devel, eval_set=[(valid, y_valid)], **fit_params)
            
            if len(model.feature_importances_) == len(features):  # some bugs in catboost?
                model.FI['fold' + str(fold_id)] = model.feature_importances_ / model.feature_importances_.sum()

            predictions = model.predict(valid)
            predictions[predictions < 0] = 0
            print("Fold ", fold_id, " error: ", mean_squared_error(y_valid, predictions)**0.5)
            
            fold_score = score(train.iloc[val], predictions)
            full_score += fold_score / len(self.fold_ids)
            print("Fold ", fold_id, " score: ", fold_score)
            
            if prepare_stacking:
                train[name].iloc[val] = np.expm1(predictions)
                
                test_predictions = model.predict(test[features])
                print(test_predictions)
                test_predictions[test_predictions < 0] = 0
                test[name] += np.expm1(test_predictions) / len(self.fold_ids)
                
        print("Final rmsle score: ", full_score)
        return full_score


## Models config

In [0]:
lgbmodel = lgb.LGBMRegressor(n_estimators=10000, 
                             objective='regression', 
                             metric='rmse',
                             max_depth = 25,
                             num_leaves=10, 
                             min_child_samples=100,
                             learning_rate=0.001,
                             boosting = 'gbdt',
                             min_data_in_leaf= 10,
                             feature_fraction = 0.2,
                             bagging_freq = 1,
                             bagging_fraction = 0.9,
                             importance_type='gain',
                             lambda_l1 = 0.2,
                             bagging_seed=42, 
                             subsample=.8, 
                             colsample_bytree=.9,
                             use_best_model=True)

xgbmodel = xgb.XGBRegressor(max_depth=10, 
                            learning_rate=0.01, 
                            n_estimators=5000, 
                            objective='reg:linear', 
                            gamma=1.45, 
                            seed=42, 
                            silent=True,
                            subsample=0.7, 
                            colsample_bytree=0.8, 
                            colsample_bylevel=0.50)

catmodel = cat.CatBoostRegressor(iterations=10000, 
                                 learning_rate=0.01, 
                                 depth=6,
                                 loss_function = "RMSE",
                                 boost_from_average = True,
                                 colsample_bylevel=0.8,
                                 bagging_temperature = 0.2,
                                 metric_period = None,
                                 random_seed=2345,
                                 l2_leaf_reg = 0.5)

In [0]:
Kfolder = KFoldValidation(train)

In [32]:
Kfolder.validate(train, test, train.columns.drop('revenue') , lgbmodel, name="lgbfinal", prepare_stacking=True) 

Fold  0 :
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's rmse: 3.07541
[200]	valid_0's rmse: 3.00622
[300]	valid_0's rmse: 2.93979
[400]	valid_0's rmse: 2.88334
[500]	valid_0's rmse: 2.83034
[600]	valid_0's rmse: 2.78185
[700]	valid_0's rmse: 2.74262
[800]	valid_0's rmse: 2.70251
[900]	valid_0's rmse: 2.66547
[1000]	valid_0's rmse: 2.63317
[1100]	valid_0's rmse: 2.60315
[1200]	valid_0's rmse: 2.5766
[1300]	valid_0's rmse: 2.55199
[1400]	valid_0's rmse: 2.53037
[1500]	valid_0's rmse: 2.50923
[1600]	valid_0's rmse: 2.49066
[1700]	valid_0's rmse: 2.47401
[1800]	valid_0's rmse: 2.45926
[1900]	valid_0's rmse: 2.44417
[2000]	valid_0's rmse: 2.43109
[2100]	valid_0's rmse: 2.41861
[2200]	valid_0's rmse: 2.40761
[2300]	valid_0's rmse: 2.39727
[2400]	valid_0's rmse: 2.38805
[2500]	valid_0's rmse: 2.37935
[2600]	valid_0's rmse: 2.3717
[2700]	valid_0's rmse: 2.36453
[2800]	valid_0's rmse: 2.35736
[2900]	valid_0's rmse: 2.35112
[3000]	valid_0's rmse: 2.34522
[3100]	v

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[13.14466969 12.45844865  5.95542504 ... 16.1563006  15.41122967
  7.1643929 ]
Fold  1 :
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's rmse: 2.75356
[200]	valid_0's rmse: 2.68617
[300]	valid_0's rmse: 2.61954
[400]	valid_0's rmse: 2.56148
[500]	valid_0's rmse: 2.50595
[600]	valid_0's rmse: 2.45614
[700]	valid_0's rmse: 2.41473
[800]	valid_0's rmse: 2.37209
[900]	valid_0's rmse: 2.33785
[1000]	valid_0's rmse: 2.30172
[1100]	valid_0's rmse: 2.26858
[1200]	valid_0's rmse: 2.23925
[1300]	valid_0's rmse: 2.21228
[1400]	valid_0's rmse: 2.18631
[1500]	valid_0's rmse: 2.16418
[1600]	valid_0's rmse: 2.14366
[1700]	valid_0's rmse: 2.12415
[1800]	valid_0's rmse: 2.10635
[1900]	valid_0's rmse: 2.0901
[2000]	valid_0's rmse: 2.07422
[2100]	valid_0's rmse: 2.05985
[2200]	valid_0's rmse: 2.04668
[2300]	valid_0's rmse: 2.03488
[2400]	valid_0's rmse: 2.02463
[2500]	valid_0's rmse: 2.015
[2600]	valid_0's rmse: 2.00546
[2700]	valid_0's rmse: 1.99685
[2800]	valid_0's rmse: 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[13.29349104 11.93289126  6.10022652 ... 15.82101279 15.05944734
  8.47345002]
Fold  2 :
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's rmse: 2.93695
[200]	valid_0's rmse: 2.85566
[300]	valid_0's rmse: 2.77261
[400]	valid_0's rmse: 2.70083
[500]	valid_0's rmse: 2.63223
[600]	valid_0's rmse: 2.57038
[700]	valid_0's rmse: 2.51889
[800]	valid_0's rmse: 2.4664
[900]	valid_0's rmse: 2.42335
[1000]	valid_0's rmse: 2.37917
[1100]	valid_0's rmse: 2.33756
[1200]	valid_0's rmse: 2.30145
[1300]	valid_0's rmse: 2.26852
[1400]	valid_0's rmse: 2.23643
[1500]	valid_0's rmse: 2.20877
[1600]	valid_0's rmse: 2.18351
[1700]	valid_0's rmse: 2.15888
[1800]	valid_0's rmse: 2.13668
[1900]	valid_0's rmse: 2.11542
[2000]	valid_0's rmse: 2.09675
[2100]	valid_0's rmse: 2.07887
[2200]	valid_0's rmse: 2.06198
[2300]	valid_0's rmse: 2.04663
[2400]	valid_0's rmse: 2.03317
[2500]	valid_0's rmse: 2.02077
[2600]	valid_0's rmse: 2.00809
[2700]	valid_0's rmse: 1.99681
[2800]	valid_0's rmse

...

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[13.21965576 12.70036724  5.99508086 ... 15.98608247 15.02182417
  6.57120086]
Fold  4 :
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's rmse: 3.10813
[200]	valid_0's rmse: 3.0334
[300]	valid_0's rmse: 2.96291
[400]	valid_0's rmse: 2.90543
[500]	valid_0's rmse: 2.85249
[600]	valid_0's rmse: 2.80111
[700]	valid_0's rmse: 2.75952
[800]	valid_0's rmse: 2.71828
[900]	valid_0's rmse: 2.68051
[1000]	valid_0's rmse: 2.64823
[1100]	valid_0's rmse: 2.61674
[1200]	valid_0's rmse: 2.59
[1300]	valid_0's rmse: 2.5668
[1400]	valid_0's rmse: 2.5439
[1500]	valid_0's rmse: 2.52347
[1600]	valid_0's rmse: 2.50599
[1700]	valid_0's rmse: 2.48846
[1800]	valid_0's rmse: 2.47384
[1900]	valid_0's rmse: 2.46098
[2000]	valid_0's rmse: 2.44957
[2100]	valid_0's rmse: 2.43846
[2200]	valid_0's rmse: 2.42827
[2300]	valid_0's rmse: 2.41869
[2400]	valid_0's rmse: 2.40978
[2500]	valid_0's rmse: 2.40201
[2600]	valid_0's rmse: 2.39532
[2700]	valid_0's rmse: 2.38763
[2800]	valid_0's rmse: 2.3

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[13.45894459 12.91719881  3.82177892 ... 16.16429337 15.23501148
  4.89709146]
Final rmsle score:  1.9588546865128458


1.9588546865128458

In [33]:
Kfolder.validate(train, test, train.columns.drop('revenue'), xgbmodel, name="xgbfinal", prepare_stacking=True)

Fold  0 :
[0]	validation_0-rmse:15.5559
Will train until validation_0-rmse hasn't improved in 500 rounds.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[100]	validation_0-rmse:6.03766
[200]	validation_0-rmse:3.05564
[300]	validation_0-rmse:2.40293
[400]	validation_0-rmse:2.30889
[500]	validation_0-rmse:2.30404
[600]	validation_0-rmse:2.30571
[700]	validation_0-rmse:2.30941
[800]	validation_0-rmse:2.30986
[900]	validation_0-rmse:2.31166
Stopping. Best iteration:
[491]	validation_0-rmse:2.30306

Fold  0  error:  2.3030592742840663
     id  transactionrevenue  predictedrevenue
0  1764          2500000.00        1214085.38
1  1162              121.00         297400.25
2   556           600000.00         199953.45
3  2008          2610000.00        2890685.50
4   993          3202000.00        1313325.50
   id  transactionrevenue  predictedrevenue
0  20         85167639.00       43388160.00
1  22         36690067.00       15760916.00
2  27         13835130.00         378030.00
3  41           163000.00        1582741.50
4  47        660940780.00      263564496.00
Fold  0  score:  2.3030592742840663


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[12.265637  11.847253   4.552079  ... 15.345253  14.673664   6.8386207]
Fold  1 :
[0]	validation_0-rmse:15.5646
Will train until validation_0-rmse hasn't improved in 500 rounds.


...

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[100]	validation_0-rmse:6.08721
[200]	validation_0-rmse:3.06475
[300]	validation_0-rmse:2.38338
[400]	validation_0-rmse:2.28036
[500]	validation_0-rmse:2.27214
[600]	validation_0-rmse:2.2724
[700]	validation_0-rmse:2.27452
[800]	validation_0-rmse:2.27672
[900]	validation_0-rmse:2.2787
[1000]	validation_0-rmse:2.27923
Stopping. Best iteration:
[568]	validation_0-rmse:2.27129

Fold  4  error:  2.2712938894513255
     id  transactionrevenue  predictedrevenue
0  2681           966878.00         741852.25
1  2578               97.00          65539.19
2   845          1012189.00        2167268.75
3   302          2850000.00         490824.84
4   822          2600000.00        5719766.50
   id  transactionrevenue  predictedrevenue
0   1         12314651.00       36211944.00
1   3         13092000.00       27269688.00
2   7         85446075.00        3783358.50
3  42         16217773.00       26689150.00
4  43         24362772.00        3485060.75
Fold  4  score:  2.2712938894513255


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[12.136235  10.334191   4.918356  ... 15.5630865 14.604865   4.245259 ]
Final rmsle score:  2.006106964650309


2.006106964650309

In [34]:
Kfolder.validate(train, test, train.columns.drop('revenue') , catmodel, name="catfinal", prepare_stacking=True,
               fit_params={"use_best_model": True, "verbose": 100})

Fold  0 :
0:	learn: 2.9913368	test: 3.1411621	best: 3.1411621 (0)	total: 5.85ms	remaining: 58.5s
100:	learn: 2.1310126	test: 2.4323452	best: 2.4323452 (100)	total: 594ms	remaining: 58.3s
200:	learn: 1.8871331	test: 2.2824697	best: 2.2824697 (200)	total: 1.12s	remaining: 54.5s
300:	learn: 1.7939621	test: 2.2474904	best: 2.2474904 (300)	total: 1.65s	remaining: 53.2s
400:	learn: 1.7373304	test: 2.2401757	best: 2.2401757 (400)	total: 2.15s	remaining: 51.6s
500:	learn: 1.6930009	test: 2.2375337	best: 2.2371763 (463)	total: 2.66s	remaining: 50.5s
600:	learn: 1.6540704	test: 2.2362235	best: 2.2361956 (586)	total: 3.17s	remaining: 49.7s
700:	learn: 1.6135982	test: 2.2364643	best: 2.2357099 (679)	total: 3.66s	remaining: 48.6s
800:	learn: 1.5729301	test: 2.2380525	best: 2.2357099 (679)	total: 4.2s	remaining: 48.2s
900:	learn: 1.5346470	test: 2.2383616	best: 2.2357099 (679)	total: 4.73s	remaining: 47.7s
1000:	learn: 1.4954429	test: 2.2404187	best: 2.2357099 (679)	total: 5.26s	remaining: 47.3s
110

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0:	learn: 3.0706895	test: 2.8129059	best: 2.8129059 (0)	total: 3.76ms	remaining: 37.6s
100:	learn: 2.2303845	test: 2.0691928	best: 2.0691928 (100)	total: 491ms	remaining: 48.2s
200:	learn: 1.9963503	test: 1.9048468	best: 1.9048468 (200)	total: 1s	remaining: 49s
300:	learn: 1.9046868	test: 1.8735125	best: 1.8733104 (299)	total: 1.46s	remaining: 46.9s
400:	learn: 1.8411450	test: 1.8658658	best: 1.8655314 (386)	total: 1.98s	remaining: 47.5s
500:	learn: 1.7962299	test: 1.8633461	best: 1.8631033 (494)	total: 2.46s	remaining: 46.7s
600:	learn: 1.7533651	test: 1.8618768	best: 1.8609207 (542)	total: 2.93s	remaining: 45.8s
700:	learn: 1.7087630	test: 1.8609633	best: 1.8607274 (697)	total: 3.4s	remaining: 45.1s
800:	learn: 1.6653738	test: 1.8631363	best: 1.8605367 (709)	total: 3.85s	remaining: 44.3s
900:	learn: 1.6226934	test: 1.8629970	best: 1.8605367 (709)	total: 4.4s	remaining: 44.4s
1000:	learn: 1.5792564	test: 1.8662267	best: 1.8605367 (709)	total: 4.95s	remaining: 44.5s
1100:	learn: 1.5353

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


100:	learn: 2.2236251	test: 2.1394452	best: 2.1394452 (100)	total: 524ms	remaining: 51.4s
200:	learn: 1.9974953	test: 1.8999537	best: 1.8999537 (200)	total: 990ms	remaining: 48.3s
300:	learn: 1.9044442	test: 1.8335199	best: 1.8335199 (300)	total: 1.56s	remaining: 50.4s
400:	learn: 1.8438110	test: 1.8122861	best: 1.8122861 (400)	total: 2.06s	remaining: 49.4s
500:	learn: 1.7948173	test: 1.8041058	best: 1.8036352 (489)	total: 2.54s	remaining: 48.2s
600:	learn: 1.7547615	test: 1.8017156	best: 1.8016957 (599)	total: 3.03s	remaining: 47.4s
700:	learn: 1.7122493	test: 1.8017510	best: 1.8003903 (682)	total: 3.53s	remaining: 46.8s
800:	learn: 1.6684823	test: 1.8000187	best: 1.7988235 (757)	total: 4s	remaining: 45.9s
900:	learn: 1.6246661	test: 1.8026196	best: 1.7988235 (757)	total: 4.52s	remaining: 45.6s
1000:	learn: 1.5810550	test: 1.8023451	best: 1.7988235 (757)	total: 5.01s	remaining: 45s
1100:	learn: 1.5392850	test: 1.8067742	best: 1.7988235 (757)	total: 5.54s	remaining: 44.8s
1200:	learn: 

...

0:	learn: 3.0389289	test: 2.9560222	best: 2.9560222 (0)	total: 5.33ms	remaining: 53.3s
100:	learn: 2.2407498	test: 2.0536803	best: 2.0536803 (100)	total: 467ms	remaining: 45.8s
200:	learn: 2.0162312	test: 1.8173608	best: 1.8173608 (200)	total: 953ms	remaining: 46.5s
300:	learn: 1.9247979	test: 1.7549622	best: 1.7548477 (299)	total: 1.43s	remaining: 46.2s
400:	learn: 1.8636215	test: 1.7416395	best: 1.7416395 (400)	total: 1.93s	remaining: 46.1s
500:	learn: 1.8108139	test: 1.7384366	best: 1.7372748 (489)	total: 2.42s	remaining: 46s
600:	learn: 1.7630250	test: 1.7374240	best: 1.7371373 (594)	total: 2.91s	remaining: 45.5s
700:	learn: 1.7196828	test: 1.7370897	best: 1.7360974 (657)	total: 3.38s	remaining: 44.8s
800:	learn: 1.6756645	test: 1.7384405	best: 1.7360974 (657)	total: 3.85s	remaining: 44.2s
900:	learn: 1.6301205	test: 1.7373485	best: 1.7360974 (657)	total: 4.35s	remaining: 44s
1000:	learn: 1.5864632	test: 1.7415558	best: 1.7360974 (657)	total: 4.87s	remaining: 43.8s
1100:	learn: 1.5

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0:	learn: 2.9810556	test: 3.1796471	best: 3.1796471 (0)	total: 15.5ms	remaining: 2m 35s
100:	learn: 2.1235665	test: 2.4743798	best: 2.4743798 (100)	total: 516ms	remaining: 50.5s
200:	learn: 1.8937732	test: 2.3284531	best: 2.3284531 (200)	total: 1.2s	remaining: 58.3s
300:	learn: 1.8089967	test: 2.2926335	best: 2.2926335 (300)	total: 1.65s	remaining: 53.3s
400:	learn: 1.7543844	test: 2.2873605	best: 2.2869551 (366)	total: 2.13s	remaining: 50.9s
500:	learn: 1.7118039	test: 2.2857305	best: 2.2850941 (489)	total: 2.58s	remaining: 49s
600:	learn: 1.6727276	test: 2.2864984	best: 2.2849862 (553)	total: 3.06s	remaining: 47.8s
700:	learn: 1.6344792	test: 2.2884172	best: 2.2849862 (553)	total: 3.53s	remaining: 46.8s
800:	learn: 1.5882985	test: 2.2903156	best: 2.2849862 (553)	total: 4.05s	remaining: 46.5s
900:	learn: 1.5426665	test: 2.2916438	best: 2.2849862 (553)	total: 4.58s	remaining: 46.2s
1000:	learn: 1.4985632	test: 2.2975290	best: 2.2849862 (553)	total: 5.04s	remaining: 45.3s
1100:	learn: 1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1.9830505531473273

# Model Stacking

In [0]:
test.head(5)

,Keywords_aftercreditsstinger,Keywords_based on novel,Keywords_biography,Keywords_duringcreditsstinger,Keywords_dystopia,Keywords_friendship,Keywords_independent film,Keywords_love,Keywords_murder,Keywords_police,Keywords_revenge,Keywords_sex,Keywords_size,Keywords_sport,Keywords_violence,Keywords_woman director,budget,cast_Bruce Willis,cast_Christopher Walken,cast_J.K. Simmons,cast_John Goodman,cast_Julianne Moore,cast_Liam Neeson,cast_Matt Damon,cast_Morgan Freeman,cast_Nicolas Cage,cast_Robert De Niro,cast_Robin Williams,cast_Samuel L. Jackson,cast_Steve Buscemi,cast_Sylvester Stallone,cast_Willem Dafoe,cast_size,crew_Avy Kaufman,crew_Deborah Aquila,crew_Francine Maisler,crew_Hans Zimmer,crew_Harvey Weinstein,crew_James Horner,crew_James Newton Howard,crew_Jerry Goldsmith,crew_Kerry Barden,crew_Luc Besson,crew_Mary Vernieu,crew_Robert Rodriguez,crew_Steven Spielberg,crew_Tricia Wood,crew_size,days_diff_from_last_movie,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_Foreign,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western,genres_size,id,last_period_mean_budget,last_period_mean_popularity,last_period_mean_revenue,last_period_median_budget,last_period_median_popularity,last_period_median_revenue,last_period_movies_count,overview_family,overview_father,overview_film,overview_finds,overview_friends,overview_help,overview_life,overview_love,overview_size,overview_story,overview_time,overview_wife,overview_woman,overview_world,overview_year,overview_years,overview_young,popularity,production_companies_Amblin Entertainment,production_companies_BBC Films,production_companies_Canal+,production_companies_Columbia Pictures,production_companies_Columbia Pictures Corporation,production_companies_Dimension Films,production_companies_DreamWorks SKG,production_companies_Dune Entertainment,production_companies_Fox 2000 Pictures,production_companies_Fox Searchlight Pictures,production_companies_Hollywood Pictures,production_companies_Lionsgate,production_companies_Metro-Goldwyn-Mayer (MGM),production_companies_Miramax Films,production_companies_New Line Cinema,production_companies_Orion Pictures,production_companies_Paramount Pictures,production_companies_Regency Enterprises,production_companies_Relativity Media,production_companies_StudioCanal,production_companies_Summit Entertainment,production_companies_Touchstone Pictures,production_companies_TriStar Pictures,production_companies_Twentieth Century Fox Film Corporation,production_companies_United Artists,production_companies_Universal Pictures,production_companies_Village Roadshow Pictures,production_companies_Walt Disney Pictures,production_companies_Warner Bros.,production_companies_Working Title Films,production_companies_size,production_countries_Australia,production_countries_Belgium,production_countries_Canada,production_countries_China,production_countries_Denmark,production_countries_France,production_countries_Germany,production_countries_Hong Kong,production_countries_India,production_countries_Ireland,production_countries_Italy,production_countries_Japan,production_countries_Mexico,production_countries_Netherlands,production_countries_Russia,production_countries_South Korea,production_countries_Spain,production_countries_Sweden,production_countries_United Kingdom,production_countries_United States of America,production_countries_size,release_date_Month,release_date_day_of_month,release_date_is_quarter_start,release_date_week_day,revenue,runtime,spoken_languages_,spoken_languages_Deutsch,spoken_languages_English,spoken_languages_Español,spoken_languages_Français,spoken_languages_Italiano,spoken_languages_Polski,spoken_languages_Português,spoken_languages_Pусский,spoken_languages_size,spoken_languages_العربية,spoken_languages_हिन्दी,spoken_languages_广州话 / 廣州話,spoken_languages_日本語,spoke

In [0]:
test['PredictedLogRevenue'] = 0.4 * test["lgbfinal"] + \
                               0.2 * test["xgbfinal"] + \
                               0.4 * test["catfinal"]



score(test, test.PredictedLogRevenue)

In [0]:
test.head(12)

In [0]:
submission = test[['id','PredictedLogRevenue']]
submission.to_csv('sub.csv')

# Charts 

In [0]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

plt.figure(figsize=(20,7))
lgb.plot_importance(lgbmodel,figsize=(10,40))
plt.yticks(fontsize=20)
plt.show()